<h1 style="text-align : center; color:red">Ropasaurusrex - PlaidCTF 2013</h1>

# Identify the main() function
We need to find the entry point of the program reading the header of the ELF file with **readelf**

<img  src="GCDITIMSGK7C8B7UR2MWW68QK8FB9LB3.png"/>

In this case the entry point is at **0x8048340**. This address usually points to the location of the first chunk of code which will be executed at run time (**_start function**) and which will handle the main function parameters (or command line arguments) before calling the main function.

We need to find the real entry point of main(). The file is stripped so we can use an heuristic to identify the entry point of the program because usually this address in a stripped file is identified by this pattern present in the **_start** function:
```
PUSH main_entry_point
CALL __libc_start_main
```
Let's see this with IDA inspecting the **_start** function:


<img  src="TTKS0HID27KYKIJD9KUJSVO6Q4HT8B6M.png"/>

So we can conclude that the main entry point is at **0x0804841d**

# Find the vulnerable function
We can start by inspecting the main() function. 

Its references are depicted in this figure:


<img  src="RRU9JEIYTY2A4T2XPD8M3693552NY81Y.png"/>

There is only one function (probably the vulnerable one) so let's inspect it. 


<img  src="U4IB3F03AIVSAU64Y34VD9HUBCJ4QN35.png"/>

This function only reads from stdin **256 bytes (100h)** and write this bytes into **buf**, but how long is buf?? we can see that the space reserved for buf is **ebp + buf --> ebp - 88h** so only 136 bytes!! (**BUFFER OVERFLOW DETECTED**).


# Setup the environment
We need to setup a proper exploiting and testing environment on our machine.
- Enable core dumping after the crash of a program 
```bash
ulimit -c unlimited
```


- Launch the executable as a service on a port, and relaunch it endlessly when crashes
```bash
while true; do nc -vv -l -p $2 -e $1; done
```


- write a simple python script to communicate with the service
```python
import socket
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.connect(("127.0.0.1", 4444))
buffer = "Hello!"
s.send(buffer)
res = s.recv(1024)
print res
```

# Crash the program
We need to find at which byte we can overwrite the return address. In order to do this we will use the utilities **pattern_reate.rb** and **pattern_offset.rb** of the metasploit-framework. 

Before 